## Análise da sequência e das features presentes no NCBI

Começamos por aceder ao NCBI para fazer download de um ficheiro no formato __genbank__ com informação correspondente à nossa zona do genoma, da posição 270001 a 505535.

O primeiro passo foi descobrir qual o __genbank_id__ associado ao accession __NC_002942.5__.
Para isto usamos a função `Entrez.search`. 

Após saber o __genbank_id__, obtivemos o __genbank__ usando a função `Entrez.efetch`.

```python
def fetch_genbank(start, end):
    """
    Procura no NCBI pelo Accession NC_002942.5 e 
    faz download de toda a informação em formato genbank,
    retornando o respectivo record com as features
    entre a posição start e end.
    """

    Entrez.email = "*@*.*"

    handle = Entrez.esearch(db="nucleotide", term="NC_002942.5")
    record = Entrez.read(handle)
    genbank_id = record["IdList"][0]
    handle.close()

    handle = Entrez.efetch(db="nucleotide",
                           rettype="gbwithparts",
                           retmod="text",
                           id=genbank_id,
                           seq_start=start,
                           seq_stop=end)
    record = SeqIO.read(handle, "gb")
    handle.close()

    return record
```

Esta função foi adicionada ao módulo `util.www`.


Deste `record` extraímos as features __gene__, __CDS__, __tRNA__ e __rRNA__.

```python
def extract_features(record):
    """
    Das várias features encontradas no ficheiro genbank, extraímos:
      - gene
      - CDS
      - tRNA
      - rRNA

    Esta função retorna uma lista apenas com as features
    dos tipos indicados acima.
    """

    result = []
    features = record.features
    feature_count = len(features)
    types = ["gene", "CDS", "tRNA", "rRNA"]

    for i in range(feature_count):
        feature = features[i]
        should_extract = feature.type in types

        if should_extract:
            result.append(feature)

    return result
```


Para finalizar, convertemos esta lista de `record` para um dicionário.
A cada um dos `record` retirámos:
- db_xref
- EC_number
- function
- gene
- gene_synonym
- locus_tag
- note
- product
- protein_id
- translation

Além disso, também ficamos com informação sobre a localização de cada gene e em qual das cadeias está presente.

```python
def features_to_dictionary(start, features):
    """
    A cada uma das features, extraimos a seguintes propriedades:
      - db_xref
      - EC_number
      - function
      - gene
      - gene_synonym
      - locus_tag
      - note
      - product
      - protein_id
      - translation

    Também extraímos a localização:
      - start
      - end
      - strand
    """
    dictionary = {}
    properties = ["db_xref",
                  "EC_number",
                  "function",
                  "gene",
                  "gene_synonym",
                  "note",
                  "product",
                  "protein_id",
                  "translation"]

    for feature in features:
        tag = feature.qualifiers["locus_tag"][0]

        if not tag in dictionary:
            # se esta tag ainda não existe,
            # criar um dicionário vazio para ela
            dictionary[tag] = {}

        location = feature.location
        dictionary[tag]["location"] = {}
        dictionary[tag]["location"]["start"] = location._start + start
        dictionary[tag]["location"]["end"] = location._end + start - 1
        dictionary[tag]["location"]["strand"] = location._strand

        if feature.type in ["tRNA", "rRNA"]:
            # Taggar as features com estes dois tipos
            dictionary[tag]["type"] = feature.type
        else:
            # se não, taggar com "mRNA"
            dictionary[tag]["type"] = "mRNA"

        for prop in properties:
            if prop in feature.qualifiers:
                value = feature.qualifiers[prop][0]

                if prop == "db_xref":
                    # renomear esta propriedade e remover "GeneID:"
                    prop = "gene_id"
                    value = value[7:]
                    
                if prop == "translation":
                    # se translation, também guardar o tamanho
                    dictionary[tag]["length"] = len(value)

                if prop in dictionary[tag]:
                    # se já encontramos esta propriedade
                    # para este locus_tag
                    # verificar que é a mesma
                    current_value = dictionary[tag][prop]
                    assert value == current_value
                else:
                    # se não, adicionar
                    dictionary[tag][prop] = value

    return dictionary
```

No final:

```python
import util.www as www

start = 270001
end = 505535

record = www.fetch_genbank(start, end)
features = extract_features(record)
dictionary = features_to_dictionary(start, features)
print(dictionary)
```

```json
{
  "lpg0232": {
    "function": "Transcription factors / DNA binding proteins",
    "gene": "np20",
    "gene_id": "19831799",
    "length": 177,
    "location": {
      "end": 270569,
      "start": 270036,
      "strand": 1
    },
    "product": "transcriptional regulator np20",
    "protein_id": "YP_094286.1",
    "translation": "MIGCCLI...",
    "type": "mRNA"
  },
  "lpg0233": {
    "EC_number": "4.1.1.7",
    "function": "Biodegradation of Xenobiotics",
    "gene": "mdlC",
    "gene_id": "19831800",
    "length": 530,
    "location": {
      "end": 272278,
      "start": 270686,
      "strand": -1
    },
    "product": "benzoylformate decarboxylase",
    "protein_id": "YP_094287.1",
    "translation": "MKKTGSDVLK...",
    "type": "mRNA"
  },
  // ...
}
```